# 🔍 Dataset Quality Filter for Japanese Keyboard IME

Filter the zenz dataset to keep high-quality, daily conversation text.

**3 Filter Layers:**
1. **Kill List** — Remove toxic/useless patterns (forum artifacts, URLs in kana, Cyrillic, etc.)
2. **Fix List** — Clean/normalize data (bad left_context, cut-off sentences)
3. **Quality Score** — Score remaining items for daily conversation relevance

**Output:** Filtered high-quality JSONL in Google Drive

In [ ]:
import os

# Auto-detect platform
if 'COLAB_RELEASE_TAG' in os.environ:
    PLATFORM = 'Colab'
    from google.colab import drive
    drive.mount('/content/drive')
    DRIVE_DIR = '/content/drive/MyDrive/Keyboard-Suggestions-ML-Colab'
elif os.path.exists('/kaggle/working'):
    PLATFORM = 'Kaggle'
    DRIVE_DIR = '/kaggle/working'
else:
    PLATFORM = 'Local'
    DRIVE_DIR = './output'

OUTPUT_DIR = f"{DRIVE_DIR}/filtered_data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"✅ Platform: {PLATFORM}")
print(f"📁 Output: {OUTPUT_DIR}")

In [ ]:
!pip install -q datasets tqdm

In [ ]:
# ===========================================================
# CONFIGURATION
# ===========================================================

# Processing limits
MAX_ITEMS = None  # None = ALL items, or e.g. 100_000 for testing

# Quality threshold (0-10 score)
MIN_QUALITY_SCORE = 4  # Keep items scoring >= this

print(f"Min quality score: {MIN_QUALITY_SCORE}")
print(f"Items limit: {'ALL' if MAX_ITEMS is None else f'{MAX_ITEMS:,}'}")

## 1. Kill List — Must Remove

These patterns destroy keyboard quality. Instant reject, no scoring needed.

In [ ]:
import re

# ==========================================================
# KILL LIST: Patterns that MUST be removed
# ==========================================================

# A. Forum / Scraping Artifacts
FORUM_PATTERNS = re.compile(
    r'(ID:|アイディ:|投稿日|投稿:|名前:|名無し|'
    r'\d{2}:\d{2}:\d{2}|'           # Timestamps 23:06:08
    r'\d{4}/\d{1,2}/\d{1,2}|'       # Dates 2010/12/15
    r'\(\w\)\s*\d{2}:\d{2}|'        # Forum date format (カ) 10:55
    r'ナマエナイヨ|'                  # Anonymous username
    r'レス数|スレッド|'               # Thread/reply terms
    r'^##\s|^■|^●|^◆)',             # Markdown/structured headers
    re.MULTILINE
)

# B. URLs spelled in Kana (useless for IME)
KANA_URL_PATTERNS = re.compile(
    r'(エイチティーティーピー|'       # http
    r'ダブリュダブリュ|'              # www
    r'ドットコム|ドットジェーピー|'    # .com, .jp
    r'ユーアールエル|'                # URL
    r'http[s]?://|'                   # Actual URLs in output
    r'www\.)',
    re.IGNORECASE
)

# C. Foreign Script Garbage (Cyrillic, extended Latin, etc.)
#    Keep basic ASCII (English is popular in Japanese!)
GARBAGE_CHARS = re.compile(
    r'[\u0400-\u04FF'     # Cyrillic
    r'\u0370-\u03FF'      # Greek (except common math)
    r'\u0100-\u024F'      # Extended Latin (Å, ö, etc.)
    r'\u0250-\u02AF'      # IPA extensions
    r'\u2600-\u26FF'      # Misc symbols (but NOT emoji)
    r']'
)

# D. ASCII art / broken encoding patterns
ASCII_ART_RE = re.compile(
    r'([─━│┃┌┐└┘├┤┬┴┼]{3,}|'   # Box drawing
    r'[=\-]{5,}|'                 # Long separators
    r'[\*]{3,}|'                  # Stars
    r'[_]{5,})',                   # Underscores
)

# E. Punctuation-only output (zero learning value)
PUNCT_ONLY_RE = re.compile(r'^[、。？！?!\s\d.,;:()（）「」『』【】・]+$')


def is_killed(item):
    """Check if item should be killed (removed). Returns (True, reason) or (False, None)."""
    inp = item.get('input', '') or ''
    out = item.get('output', '') or ''
    ctx = item.get('left_context', '') or ''
    full = ctx + out
    
    # Empty
    if not inp or not out:
        return True, 'empty'
    
    # Forum artifacts
    if FORUM_PATTERNS.search(full):
        return True, 'forum_artifact'
    
    # Kana URLs
    if KANA_URL_PATTERNS.search(inp) or KANA_URL_PATTERNS.search(out):
        return True, 'kana_url'
    
    # Foreign script garbage (NOT basic ASCII — English is OK!)
    if GARBAGE_CHARS.search(out):
        return True, 'garbage_chars'
    
    # ASCII art / broken encoding
    if ASCII_ART_RE.search(out):
        return True, 'ascii_art'
    
    # Punctuation-only output
    if PUNCT_ONLY_RE.match(out):
        return True, 'punct_only'
    
    return False, None


# Quick test
print("🧪 Kill List test:")
kill_tests = [
    {'input': 'ナマエ', 'output': '名前: テスト ID:123', 'left_context': ''},
    {'input': 'エイチティーティーピーエス', 'output': 'https://example.com', 'left_context': ''},
    {'input': 'テスト', 'output': 'ЅМАЯ', 'left_context': ''},
    {'input': 'テン', 'output': '、', 'left_context': ''},
    {'input': 'きょう', 'output': '今日はいい天気ですね', 'left_context': ''},
    {'input': 'アプリ', 'output': 'appを使って', 'left_context': ''},  # English OK!
]
for t in kill_tests:
    killed, reason = is_killed(t)
    status = f'❌ KILL ({reason})' if killed else '✅ KEEP'
    print(f"  {status}: {t['input']} → {t['output'][:30]}")

## 2. Fix List — Clean & Normalize

Don't delete these — fix them to make the IME smarter.

In [ ]:
# ==========================================================
# FIX LIST: Clean/normalize data
# ==========================================================

# Bad left_context patterns (set to empty)
BAD_CONTEXT_PATTERNS = re.compile(
    r'(^ID:|^アイディ:|^投稿日|^名前:|'
    r'^\d{4}[/\-]\d{1,2}[/\-]\d{1,2}|'   # Date-only context (2023/10/01)
    r'^\d{2}:\d{2}:\d{2}|'                  # Time-only context
    r'http[s]?://|'                          # URL context
    r'^[A-Z]{5,}$)',                         # All-caps codes
    re.IGNORECASE
)

# Particles that can't start a sentence without context
STARTING_PARTICLES = {'て', 'に', 'を', 'は', 'が', 'で', 'と', 'も',
                       'へ', 'から', 'まで', 'より', 'ので', 'のに',
                       'けど', 'けれど', 'ながら'}


def fix_item(item):
    """Clean and normalize an item. Returns fixed item."""
    result = {
        'input': item.get('input', '') or '',
        'output': item.get('output', '') or '',
        'left_context': item.get('left_context', '') or '',
    }
    
    ctx = result['left_context']
    out = result['output']
    
    # Fix 1: Clean bad left_context → set to empty
    if ctx and BAD_CONTEXT_PATTERNS.search(ctx):
        result['left_context'] = ''
        result['_fixed'] = 'bad_context_cleared'
    
    # Fix 2: Cut-off sentences — output starts with particle but no context
    #         "て継続して" with no context → meaningless fragment
    if not result['left_context']:
        first_chars = out[:3]
        if any(first_chars.startswith(p) for p in STARTING_PARTICLES):
            # This is a cut-off fragment without context
            result['_cutoff'] = True
    
    # Fix 3: Strip whitespace
    result['left_context'] = result['left_context'].strip()
    result['output'] = result['output'].strip()
    result['input'] = result['input'].strip()
    
    return result


# Quick test
print("🧪 Fix List test:")
fix_tests = [
    {'input': 'アメ', 'output': '雨だった', 'left_context': '2023/10/01'},
    {'input': 'ケイゾク', 'output': 'て継続して', 'left_context': ''},
    {'input': 'アメ', 'output': '雨だった', 'left_context': '昨日は'},
]
for t in fix_tests:
    fixed = fix_item(t)
    ctx_before = t['left_context'] or '(none)'
    ctx_after = fixed['left_context'] or '(none)'
    flags = []
    if fixed.get('_fixed'): flags.append(f"🔧 {fixed['_fixed']}")
    if fixed.get('_cutoff'): flags.append('⚠️ cutoff')
    flag_str = ' '.join(flags) if flags else '✅ OK'
    print(f"  {flag_str}: ctx [{ctx_before}] → [{ctx_after}] | {fixed['output'][:20]}")

## 3. Quality Score — Daily Conversation Relevance

Score 0-10 based on how useful this item is for keyboard IME.

In [ ]:
from collections import Counter

# ==========================================================
# Daily vocabulary (for positive scoring)
# ==========================================================

# Common daily verbs (stem forms to match conjugations)
DAILY_VERBS = {
    'する', 'し', 'さ', 'いる', 'い', 'ある', 'あっ', 'なる', 'なっ',
    'できる', 'でき', 'やる', 'やっ',
    '行く', '行き', '行っ', '来る', '来た', '帰る', '帰っ',
    '見る', '見た', '見え', '聞く', '聞い', '読む', '読ん',
    '食べ', '飲む', '飲ん', '買う', '買っ', '作る', '作っ',
    '使う', '使っ', '持つ', '持っ', '入る', '入っ', '出る', '出し',
    '思う', '思っ', '考え', '知る', '知っ', '分かる', '分かっ',
    '言う', '言っ', '話す', '話し', '書く', '書い', '教え',
    '住む', '住ん', '働く', '働い', '始め', '終わ',
    '待つ', '待っ', '送る', '送っ', '受け', '取る', '取っ',
    '遊ぶ', '遊ん', '走る', '走っ', '歩く', '歩い',
    '起き', '寝る', '寝た', '開く', '開い', '閉め',
    '変わる', '変わっ', '決め', '選ぶ', '選ん',
    '会う', '会っ', '呼ぶ', '呼ん', '払う', '払っ',
    '立つ', '立っ', '座る', '座っ', '置く', '置い',
    '好き', '嫌い', '欲しい', '楽しい', '嬉しい',
    '必要', '大切', '大事', '簡単', '難しい',
}

# Common daily nouns
DAILY_NOUNS = {
    '人', '事', '物', '所', '方', '時', '日', '年', '月', '前',
    '後', '中', '上', '下', '間', '気', '目', '手', '頭', '心',
    '名前', '仕事', '学校', '会社', '家', '部屋', '電車', '駅',
    '友達', '家族', '子供', '先生', '学生', '彼', '彼女', '自分',
    '朝', '昼', '夜', '今日', '明日', '昨日', '今', '週末',
    '天気', '雨', '雪', '春', '夏', '秋', '冬',
    '食事', '料理', '水', 'お茶', 'コーヒー', 'ご飯',
    '電話', 'メール', '写真', '映画', '音楽', '本', 'ゲーム',
    '店', '病院', '公園', '道', '町', '国', '場所',
    '問題', '質問', '答え', '意味', '理由', '結果',
    '話', '言葉', '声', '体', '顔', '口',
    '車', '花', '猫', '犬', '空', '海', '山',
    'お金', '時間', '趣味', '経験', '気持ち', '生活',
    'スマホ', 'アプリ', 'プログラム', 'データ', 'サイト',
}

# Polite/conversational markers
POLITE_MARKERS = {
    'です', 'ます', 'でした', 'ました', 'ません', 'ください',
    'でしょう', 'ましょう', 'ですか', 'ますか',
    'だ', 'だった', 'だろう', 'かな', 'よね', 'ね', 'よ',
    'けど', 'けれど', 'ので', 'から', 'たら', 'ても',
    'なりました', 'ありがとう', 'すみません', 'お願い',
}

# Encyclopedia patterns (reduce score)
ENCYCLOPEDIA_PATTERNS = re.compile(
    r'(に位置する|に所在する|'          # Location descriptions
    r'北緯\d+|南緯\d+|東経\d+|西経\d+|'  # Coordinates
    r'ISBN|ISSN|'                        # Book codes
    r'紀元前\d|'                         # BC dates
    r'人口は約?\d|面積は約?\d|'          # Population/area stats
    r'標高\d|海抜\d|'                    # Elevation
    r'学名|分類学|'                      # Scientific taxonomy
    r'第\d+代|第\d+回|'                  # Ordinal titles
    r'条約|勅令|法令)',                   # Legal terms
)

# Number density pattern
NUMBER_RE = re.compile(r'\d+')


def score_quality(item):
    """Score an item 0-10 for daily conversation relevance.
    
    Note: item should already be passed through fix_item().
    Returns: (score, reasons_list)
    """
    ctx = item.get('left_context', '') or ''
    inp = item.get('input', '') or ''
    out = item.get('output', '') or ''
    full_text = ctx + out
    
    score = 5  # Start neutral
    reasons = []
    
    # --- NEGATIVE SIGNALS ---
    
    # Cut-off fragment (particle start without context)
    if item.get('_cutoff'):
        score -= 2
        reasons.append('cutoff_fragment')
    
    # Very long text (likely encyclopedia paragraphs)
    if len(full_text) > 140:
        score -= 1
        reasons.append('very_long')
    
    # Encyclopedia patterns
    if ENCYCLOPEDIA_PATTERNS.search(full_text):
        score -= 2
        reasons.append('encyclopedia')
    
    # High number density (3+ number groups = statistics)
    numbers = NUMBER_RE.findall(full_text)
    if len(numbers) >= 3:
        score -= 2
        reasons.append(f'many_numbers({len(numbers)})')
    elif len(numbers) >= 2:
        score -= 1
        reasons.append(f'numbers({len(numbers)})')
    
    # Long consecutive numbers (serial codes, etc.)
    if re.search(r'\d{5,}', full_text):
        score -= 2
        reasons.append('long_number')
    
    # Many brackets (technical notation)
    brackets = sum(full_text.count(c) for c in '(（[【')
    if brackets >= 3:
        score -= 1
        reasons.append(f'brackets({brackets})')
    
    # --- POSITIVE SIGNALS ---
    
    # Contains daily verbs
    daily_verb_count = sum(1 for v in DAILY_VERBS if v in full_text)
    if daily_verb_count >= 2:
        score += 2
        reasons.append(f'daily_verbs({daily_verb_count})')
    elif daily_verb_count >= 1:
        score += 1
        reasons.append('daily_verb')
    
    # Contains daily nouns
    daily_noun_count = sum(1 for n in DAILY_NOUNS if n in full_text)
    if daily_noun_count >= 2:
        score += 1
        reasons.append(f'daily_nouns({daily_noun_count})')
    
    # Polite/conversational markers
    polite_count = sum(1 for p in POLITE_MARKERS if p in full_text)
    if polite_count >= 1:
        score += 1
        reasons.append(f'polite({polite_count})')
    
    # Natural sentence ending
    if full_text and full_text[-1] in '。！？ね、よか':
        score += 1
        reasons.append('natural_end')
    
    # Good output length (natural phrase)
    if 3 <= len(out) <= 50:
        score += 1
        reasons.append('good_out_len')
    
    # Has meaningful left_context (provides training signal)
    if len(ctx) >= 2:
        score += 1
        reasons.append('has_context')
    
    # Mixed script (natural Japanese + katakana/English)
    has_hiragana = bool(re.search(r'[\u3040-\u309F]', full_text))
    has_katakana = bool(re.search(r'[\u30A0-\u30FF]', full_text))
    has_kanji = bool(re.search(r'[\u4E00-\u9FFF]', full_text))
    if has_hiragana and has_kanji:
        score += 1
        reasons.append('mixed_script')
    
    # Clamp to 0-10
    score = max(0, min(10, score))
    
    return score, reasons


# Quick test
print("🧪 Quality Score test:")
score_tests = [
    {'input': 'きょう', 'output': '今日はいい天気ですね', 'left_context': ''},
    {'input': 'ほくい', 'output': '北緯35度12分に位置する', 'left_context': ''},
    {'input': 'いっしょ', 'output': '一緒に映画を見に行きました', 'left_context': '友達と'},
    {'input': 'し', 'output': 'て継続して', 'left_context': '', '_cutoff': True},
    {'input': 'アプリ', 'output': 'appを使って見る', 'left_context': 'スマホの'},
    {'input': 'だい', 'output': '第35代大統領の', 'left_context': ''},
    {'input': 'あつ', 'output': '暑い', 'left_context': '今日はとても'},  # Short output OK!
]
for t in score_tests:
    s, r = score_quality(t)
    status = '✅' if s >= MIN_QUALITY_SCORE else '❌'
    ctx = t.get('left_context', '')[:10] or ''
    print(f"  {status} Score {s:2d}/10 | {ctx}<SEP>{t['input']} → {t['output'][:25]}")
    print(f"              Reasons: {', '.join(r)}")

## 4. Load Dataset & Process

In [ ]:
from datasets import load_dataset
from tqdm.auto import tqdm
import json

print("📥 Loading zenz dataset...")
dataset = load_dataset(
    "Miwa-Keita/zenz-v2.5-dataset",
    data_files="train_wikipedia.jsonl",
    split="train"
)
total = len(dataset)
print(f"✓ Loaded {total:,} items")

In [ ]:
# ============================================================
# Full pipeline: Kill → Fix → Score → Save
# ============================================================

FILTERED_FILE = f"{OUTPUT_DIR}/filtered_high_quality.jsonl"
KILLED_FILE = f"{OUTPUT_DIR}/killed_samples.jsonl"  # Sample of removed items
PROGRESS_FILE = f"{OUTPUT_DIR}/filter_progress.json"

# Stats
stats = {
    'total': 0,
    'killed': Counter(),       # Reason → count
    'score_dist': Counter(),   # Score → count
    'kept': 0,
    'fixed': Counter(),        # Fix type → count
}

# Resume support
start_from = 0
if os.path.exists(PROGRESS_FILE):
    with open(PROGRESS_FILE, 'r') as f:
        progress = json.load(f)
    start_from = progress.get('processed', 0)
    stats['kept'] = progress.get('kept', 0)
    stats['killed'] = Counter(progress.get('killed', {}))
    stats['score_dist'] = Counter({int(k): v for k, v in progress.get('score_dist', {}).items()})
    stats['fixed'] = Counter(progress.get('fixed', {}))
    print(f"📂 Resuming from item {start_from:,}")

limit = min(total, MAX_ITEMS) if MAX_ITEMS else total

# Keep some killed samples for inspection
killed_samples = []
MAX_KILLED_SAMPLES = 100

# Process
mode = 'a' if start_from > 0 else 'w'
with open(FILTERED_FILE, mode, encoding='utf-8') as out_f:
    for idx in tqdm(range(start_from, limit), desc="Filtering", total=limit - start_from):
        item = dataset[idx]
        stats['total'] += 1
        
        # Step 1: KILL LIST — instant reject
        killed, kill_reason = is_killed(item)
        if killed:
            stats['killed'][kill_reason] += 1
            if len(killed_samples) < MAX_KILLED_SAMPLES:
                killed_samples.append({
                    'reason': kill_reason,
                    'input': item.get('input', '')[:30],
                    'output': (item.get('output', '') or '')[:50],
                })
            continue
        
        # Step 2: FIX LIST — clean/normalize
        fixed = fix_item(item)
        if fixed.get('_fixed'):
            stats['fixed'][fixed['_fixed']] += 1
        if fixed.get('_cutoff'):
            stats['fixed']['cutoff'] += 1
        
        # Step 3: QUALITY SCORE
        score, reasons = score_quality(fixed)
        stats['score_dist'][score] += 1
        
        if score >= MIN_QUALITY_SCORE:
            out_item = {
                'left_context': fixed['left_context'],
                'input': fixed['input'],
                'output': fixed['output'],
                'score': score,
            }
            out_f.write(json.dumps(out_item, ensure_ascii=False) + '\n')
            stats['kept'] += 1
        
        # Save progress every 100K items
        if stats['total'] % 100_000 == 0:
            with open(PROGRESS_FILE, 'w') as pf:
                json.dump({
                    'processed': start_from + stats['total'],
                    'kept': stats['kept'],
                    'killed': dict(stats['killed']),
                    'score_dist': dict(stats['score_dist']),
                    'fixed': dict(stats['fixed']),
                }, pf)
            out_f.flush()

# Save final progress
with open(PROGRESS_FILE, 'w') as pf:
    json.dump({
        'processed': start_from + stats['total'],
        'kept': stats['kept'],
        'killed': dict(stats['killed']),
        'score_dist': dict(stats['score_dist']),
        'fixed': dict(stats['fixed']),
    }, pf)

# Save killed samples
with open(KILLED_FILE, 'w', encoding='utf-8') as f:
    for s in killed_samples:
        f.write(json.dumps(s, ensure_ascii=False) + '\n')

total_processed = start_from + stats['total']
total_killed = sum(stats['killed'].values())

print(f"\n✓ Processed: {total_processed:,}")
print(f"🗑️ Killed: {total_killed:,}")
print(f"✅ Kept: {stats['kept']:,} ({stats['kept']/total_processed*100:.1f}%)")
print(f"💾 Saved: {FILTERED_FILE}")

## 5. Results & Statistics

In [ ]:
total_processed = start_from + stats['total']
total_killed = sum(stats['killed'].values())
total_scored = total_processed - total_killed

print("="*60)
print("📊 FILTERING REPORT")
print("="*60)

# Kill List breakdown
print(f"\n🗑️ KILL LIST ({total_killed:,} removed):")
print("-" * 40)
for reason, count in stats['killed'].most_common():
    pct = count / total_processed * 100
    print(f"  {reason:<20} {count:>8,} ({pct:.1f}%)")

# Fix List breakdown
total_fixed = sum(stats['fixed'].values())
print(f"\n🔧 FIX LIST ({total_fixed:,} items cleaned):")
print("-" * 40)
for fix_type, count in stats['fixed'].most_common():
    print(f"  {fix_type:<20} {count:>8,}")

# Score distribution
print(f"\n📈 SCORE DISTRIBUTION (of {total_scored:,} non-killed items):")
print("-" * 55)
for score in sorted(stats['score_dist'].keys()):
    count = stats['score_dist'][score]
    pct = count / max(total_scored, 1) * 100
    bar = '█' * int(pct / 2)
    status = '✅ KEEP' if score >= MIN_QUALITY_SCORE else '❌ DROP'
    print(f"  Score {score:2d}: {count:>8,} ({pct:5.1f}%) {bar} {status}")

# Summary
print(f"\n{'='*60}")
print(f"📋 SUMMARY")
print(f"  Original:     {total_processed:>10,}")
print(f"  Killed:       {total_killed:>10,} ({total_killed/total_processed*100:.1f}%)")
print(f"  Score < {MIN_QUALITY_SCORE}:    {total_scored - stats['kept']:>10,}")
print(f"  ✅ Kept:      {stats['kept']:>10,} ({stats['kept']/total_processed*100:.1f}%)")

file_size = os.path.getsize(FILTERED_FILE)
print(f"\n  File: {FILTERED_FILE}")
print(f"  Size: {file_size / (1024*1024):.1f} MB")

In [ ]:
# Show killed samples for verification
print("\n🗑️ Sample KILLED items (verify these should be removed):")
print("-" * 60)
# Group by reason
by_reason = {}
for s in killed_samples:
    r = s['reason']
    if r not in by_reason:
        by_reason[r] = []
    if len(by_reason[r]) < 3:  # 3 samples per kill reason
        by_reason[r].append(s)

for reason, samples in by_reason.items():
    print(f"\n  [{reason}]")
    for s in samples:
        print(f"    {s['input']} → {s['output'][:40]}")

In [ ]:
# Show kept samples for verification
print("\n✅ Sample KEPT items (verify these are good quality):")
print("-" * 60)

kept_samples = []
with open(FILTERED_FILE, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 30:
            break
        kept_samples.append(json.loads(line))

for item in kept_samples[:15]:
    ctx = item['left_context'][:12] or ''
    print(f"  s={item['score']} | {ctx}<SEP>{item['input'][:10]} → {item['output'][:30]}")

In [ ]:
# How to use in training notebooks
print("\n" + "="*60)
print("📋 HOW TO USE IN TRAINING NOTEBOOKS")
print("="*60)
print(f"""
Replace dataset loading with:

  import json
  
  FILTERED_FILE = "{FILTERED_FILE}"
  
  dataset = []
  with open(FILTERED_FILE, 'r', encoding='utf-8') as f:
      for line in f:
          dataset.append(json.loads(line))
  
  print(f"Loaded {{len(dataset):,}} filtered items")
  
  # Same fields as before:
  for item in dataset:
      left_ctx = item['left_context']
      kana = item['input']
      kanji = item['output']
""")